pandas project

índice:
1. importación
2. exploración de los datos
3. toma de decisiones
4. data cleaning
5. análisis

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = "ISO-8859-1")
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
len(data)

5992

la importación daba un error:
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1: invalid start byte
Busqué este error y di con la solución en stackoverflow, que era añadir 'encoding = "ISO-8859-1"' como argumento al llamar a pd.read_csv

In [4]:
#vemos las columnas que tenemos y cuántos valores null tienen
null_cols = data.isnull().sum()
null_cols

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

columnas Unnamed:22 y Unnamed:23 son todo nulls, por lo que habría que eliminarlas

columnas con información redundante, como las fechas. nos valdría con Year, no necesitamos Date

de la misma manera, con Country nos vale, no necesitamos Area ni Location

podríamos hacer estudio por países. en la gráfica mostraría solo los países en los que más ataques se producen
el campo Activity me parece interesante

voy a conservar el campo Fatal (Y/N)

estudiaremos solamente un periodo de varias décadas

estaría bien usar RegEx

nos quedamos con el Case Number, lo usaremos como id del evento

función apply

eliminar columnas con más de 2500 nulls

el campo Sex tiene un espacio. strip todos los campos

campo activity, agrupar fishing

surf skiing tiene dos nombres

In [5]:
data.groupby("Type").count()[["Case Number"]]

,Case Number
Type,
Boat,200
Boating,110
Invalid,519
Provoked,557
Sea Disaster,220
Unprovoked,4386


In [6]:
#vemos que tenemos boat y boating, así que unificaremos las dos

In [7]:
data.groupby("Year").count()[["Case Number"]].head()

,Case Number
Year,
0,124
5,1
77,1
500,1
1543,1


In [8]:
data.groupby("Country").count()[["Case Number"]].sort_values('Case Number', ascending=False).head()

,Case Number
Country,
USA,2116
AUSTRALIA,1279
SOUTH AFRICA,565
PAPUA NEW GUINEA,133
NEW ZEALAND,125


In [9]:
data.groupby("Sex ").count()[["Case Number"]].sort_values('Case Number', ascending=False)

,Case Number
Sex,
M,4835
F,585
M,2
.,1
N,1
lli,1


In [10]:
data.groupby("Activity").count()[["Case Number"]].sort_values('Case Number', ascending=False).head()

,Case Number
Activity,
Surfing,904
Swimming,819
Fishing,414
Spearfishing,321
Bathing,153


In [11]:
data.groupby("Fatal (Y/N)").count()[["Case Number"]].sort_values('Case Number', ascending=False).head()

,Case Number
Fatal (Y/N),
N,4315
Y,1552
UNKNOWN,94
N,8
#VALUE!,1


ya sabemos los campos con los que nos vamos a quedar:

1. Case Number: id
2. Year: unidad temporal
3. Type: tipología del accidente
4. Country: país donde se dio el accidente
5. Activity: actividad que estaba haciendo la víctima
6. Sex: sexo de la víctima
7. Fatal (Y/N): fallecimiento o no de la víctima

pasos a seguir:
1. strip nombres de las columnas
2. corregir tipo de datos
3. strip columnas que sean strings
4. eliminar duplicados
5. eliminar las columnas con más de 2500 nulls
6. eliminar columnas que no vamos a utilizar
7. cambiar orden de las columnas
8. eliminar los años anteriores a 1920

He detectado que algunos nombres de las columnas tienen espacios al principio y/o al final, por lo que aplico strip(). Además, los pongo en minúsculas para que sea más cómodo escribir sus nombres.

In [12]:
data.columns = data.columns.str.strip().str.lower()
data.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

eliminar duplicados

In [13]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

Number of duplicate records dropped:  0


eliminar columnas con más de 2500 nulls

In [ ]:
drop_cols = list(null_cols[null_cols > 2500].index)
data = data.drop(drop_cols, axis=1)